In [1]:
# import libraries
from itertools import chain
import numpy as np
np.set_printoptions(precision = 4, suppress = True)
import pandas as pd
import os
from datetime import datetime
import mplfinance as mpf
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from matplotlib.dates import num2date, date2num
%matplotlib qt

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
out_data_dir = "../data/out"
in_data_dir = "../data/in"

In [4]:
def get_data_points(csv_path, sep=",", date_col="time"):    
    # read file
    data_points = pd.read_csv(csv_path, sep)
    
    # convert time
    data_points[date_col] = data_points[date_col].apply(lambda x: datetime.utcfromtimestamp(x))
    
    # set index
    data_points.set_index(date_col, inplace=True)

    return data_points

In [5]:
open_points = get_data_points(os.path.join(out_data_dir, "open_points.csv"))
close_points = get_data_points(os.path.join(out_data_dir, "closed_points.csv"))
mean_price_points = get_data_points(os.path.join(out_data_dir, "mean_price_points.csv"))
indics_points = get_data_points(os.path.join(out_data_dir, "indicator_values.csv"))

In [6]:
ohlc = get_data_points(os.path.join(in_data_dir, "btc-usdt-30-min.csv"), sep=";")

In [7]:
# reindex
open_points = open_points.reindex(ohlc.index)
closed_points = close_points.reindex(ohlc.index)
mean_price_points = mean_price_points.reindex(ohlc.index)
indics_points = indics_points.reindex(ohlc.index)

In [8]:
pair = "BTC/USDT"
interval = "30 min"
title = f"{pair}\n{interval}"

candlesticks_labels = ['candlestick wick', 'candlestick body']
indic_vals_labels = list(indics_points)
mean_price_labels = list(mean_price_points)
open_points_label = ["open position"]
close_points_label = ["closed position"]

labels = list(chain(candlesticks_labels, indic_vals_labels, mean_price_labels, open_points_label, close_points_label))

In [9]:
def plot_data_points(ax, min_time, max_time):
    addons = [
        mpf.make_addplot(indics_points.loc[min_time:max_time], ax=ax),
        mpf.make_addplot(mean_price_points.loc[min_time:max_time], ax=ax),
        mpf.make_addplot(open_points.loc[min_time:max_time], ax=ax, type='scatter', markersize=200, marker='^'),
        mpf.make_addplot(closed_points.loc[min_time:max_time], ax=ax, type='scatter', markersize=200, marker='v'),
    ]
    ax.clear()
    mpf.plot(ohlc.loc[min_time:max_time], ax=ax, addplot=addons, type='candle', style='binance')
    ax.legend(labels, loc='center left', bbox_to_anchor=(1, 0.5))

In [10]:
# https://stackoverflow.com/questions/31015755/datetime-with-slider-widget-in-matplotlib
# https://stackoverflow.com/questions/71164033/date-tick-marks-mm-dd-on-slider
# https://stackoverflow.com/questions/4700614/how-to-put-the-legend-outside-the-plot
n_points = 250

min_time = ohlc.index[0]
max_time = ohlc.index[n_points]
delta_time = max_time - min_time

# create figure
fig = mpf.figure(style='binance', figsize=(10, 6))
fig.suptitle(title)
ax = fig.add_subplot(1,1,1)
plt.subplots_adjust(bottom=0.25) # add space for slider

# shrink current axis to fit legend
box = ax.get_position()
shrink_ratio = 0.85
ax.set_position([box.x0, box.y0, box.width * shrink_ratio, box.height])

# create slider
slider_ax = plt.axes([0.18, 0.05, 0.6, 0.03])
slider_max_index = len(ohlc.index) - n_points - 1
slider_max_time = ohlc.index[slider_max_index]
slider_min_val = date2num(min_time)
slider_max_val = date2num(slider_max_time)
time_slider = Slider(slider_ax, "time", slider_min_val, slider_max_val)

# add time ticks
slider_ax.add_artist(slider_ax.xaxis)
time_tick_vals = np.linspace(slider_min_val, slider_max_val, 8)
slider_ax.set_xticks(time_tick_vals, [num2date(s).strftime("%m-%d-%y") for s in time_tick_vals])

# update figure based on slider value
def update(min_pos):
    min_time = num2date(min_pos)
    max_time = num2date(min_pos) + delta_time
    plot_data_points(ax, min_time, max_time)
    time_slider.valtext.set_text(min_time.strftime("%m/%d/%Y, %H:%M:%S"))
    fig.canvas.draw_idle()

# init time slider
update(slider_min_val)
time_slider.on_changed(update)

0